# Tutorial_1 Das BWKI Hundetutorial 🐶🩺

## Musterlösung für Kapitel 4 📚

Dies ist die Musterlösung zu den Programmieraufgaben des vierten Kapitels - einem sehr mathematischen Kapitel.

Schaut sie euch erst an wenn ihr versucht habt, die Übung selbstständig zu lösen und nicht mehr weiter kommt. Bei Fragen könnt ihr euch jederzeit auf unserem Discord Server mit anderen TeilnehmerInnen austauschen!

Viel Erfolg bei diesem Kapitel!

Als erstes müssen wir unsere **Daten einlesen** und für die Datenverarbeitung benötigte Packages (Module) importieren.

### Vorbereitung 

In [1]:
#Importiere numpy, ein Paket zur Datenverarbeitung
import numpy as np

In [2]:
#Prüfe in welchem Arbeitsverzeichnis (working directory) du gerade arbeitest 
#Im selben Ordner müssen auch die Daten gespeicherts sein
%pwd

'/home/jovyan/ML/Erkranktet Hunde'

In [ ]:
#Du bist noch nicht im richtigen Ordner? Ändere mit cd / deine working directory
cd /Users/....

In [3]:
#Einlesen der Daten
D = np.load('train_data.npy')


In [4]:
# Eigenschaften des Datensatzes einsehen (Zeilen, Spalten)
D.shape

(426, 14)

### Was bisher geschah ....

In den vorangegangenen Kapiteln hast du gelernt einzelne Datenpunkte aus einem Datensatz zu extrahieren, kannst for und while Schleifen, if-Abfragen und Funktionen der numpy-Bibliothek anwenden. Zusätzlich hast du gelernt, wie du einen Klassifikator mit ein oder mehreren Merkmalen implementierst und wie du überprüfst, ob der Klassifikator eine hohe Genauigkeit für seine Zuordnungen aufweist. In den folgenden Aufgaben wirst du lernen, wie dein Algorithmus von selbst herausfindet welche Gewichte zum besten Ergebnis führen - d.h. kein Ausprobieren mehr, sondern die Anwendung des **Gradientenverfahren**. 

In der ersten Übung ist es an der Zeit eine Kostenfunktion, auf Basis des natürlichen Logarithmus, zu implementieren. Die Kostenfunktion gibt dir ein Feedback, wie "schlimm" die Fehler in den Vorhersagen unseres Algorithmus waren. <br> **Wichtig:** Die Kostenfunktion gibt einen kleinen Wert zurück, wenn unsere Vorhersage (fast) richtig ist und einen großen Wert, wenn sie (sehr) daneben liegt. 

Deine Aufgabe ist es die Kostenfunktion zu implementieren und den Mittelwert aller Kosten zurück zu geben (Abschnitt 5/15). Dies ist die MUSTERLÖSUNG! Die Coding-Konsole im KI-Kurs enthält weitere Informationen zu Funktionen, Argumenten oder Erklärungen zum Ergebnis.

In [5]:
#import numpy as np

#definiere cost = Kostenfunktion
def cost(predictions, labels):
    """Kostenfunktion.
    
    Args:
        predictions (numpy.ndarray): Array mit den Wahrscheinlichkeiten für jeden Hund
        labels (numpy.ndarray): Array mit den Labels (0 oder 1) für jeden Hund
    Returns:
        float: je kleiner der Wert desto besser ist die Vorhersage des Klassifikators
    """
    # Abfangen eines Sonderfalls: Der Logarithmus ist an der Stelle 0 nicht definiert.
    epsilon = 1e-12
    predictions[predictions <= epsilon] = epsilon
    predictions[predictions >= 1. - epsilon] = 1. - epsilon

    cost = np.mean(-(labels*np.log(predictions)
             + (1-labels)*np.log(1-predictions)))
    return cost

Super - Aufgabe gelöst!
Die obige Funktion ist vorerst ein Bauplan, ein Test ob diese funktioniert ist jedoch mit ein paar Tricks möglich!

---

### Tipps & Tricks (nicht für Lösung im KI Kurs notwendig)
Wir können cost zwar nicht "einfach so" testen, aber da das ja eine ganz allgemeine Kostenfunktion ist,
können wir uns für den Test temporär eigene Daten in einem numpy.array definieren & cost auf diesen Daten ausführen.
Das geht z.B. so:

In [6]:
#Unser temporärer Datensatz
labels_temporary = np.array([1.0,
                            0.0,
                            1.0,
                            0.0])

#Wir denken uns einige stark abweichende Vorhersagen aus
preds_bad_temporary = np.array([0.7,
                               0.4,
                               0.0,
                               0.9])

#Wir denken uns einige gut passende Vorhersagen aus
preds_good_temporary = np.array([0.9,
                                0.1,
                                0.9,
                                0.0])

#Wir wenden die Kostenfunktion an und berechnen wie ungünstig die Fehler in der Vorhersage sind 
cost_bad = cost(preds_bad_temporary,labels_temporary)
cost_good = cost(preds_good_temporary,labels_temporary)

#Wir lassen uns das Ergebnis ausgeben
print("Kosten der schlechten Vorhersagen: {}\nKosten der guten Vorhersagen: {}\n".format(cost_bad,cost_good))

Kosten der schlechten Vorhersagen: 7.7002766941568295
Kosten der guten Vorhersagen: 0.0790203867436197



Dies sind komplett zufällig ausgedachte Daten, die wir einfach nur zu Testzwecken verwenden.
Wie wir sehen, gibt die Kostenfunktion einen höheren Wert zurück,
wenn unsere Vorhersagen weiter weg von den tatsächlichen Labels der Originaldaten liegen.

---

Um zu berechnen wann die Kostenfunktion minimal ist, wenden wir das Gradientenverfahren an. Um das Minimum zu bestimmen, verwenden wir die partielle Ableitung. Deine Aufgabe ist es die Funktion derivative, die die partielle Ableitung der Kostenfunktion nach den Gewichten w1, w2 und dem Bias b berechnet, zu implementieren und die Funktion p() anzupassen (Abschnitt 11/15). Dies ist die MUSTERLÖSUNG! Die Coding-Konsole im KI-Kurs enthält weitere Informationen zu Funktionen, Argumenten oder Erklärungen zum Ergebnis.

In [7]:
#import numpy as np

#Zuerst müssen wir einige Funktionen implementieren, die wir schon aus den vorherigen Kapiteln kennen

# Um die Wahrscheinlichkeit für eine Zuordnung zu berechnen (Wie sicher ist sich der Algorithmus?)
def sigmoid(x):
    """Sigmoid Funktion
    Args:
        x (np.ndmatrix): Werte auf den die Sigmoid-Funktion angewendet wird
    Returns:
         np.array: Transformierte Werte.
    Note:
        Verwende die np.exp Funktion. Alle Funktionen die du hier verwendest
        aktzeptieren auch Vektoren. Somit unterscheidet sich die
        Vektor-Implementierung nicht von der Skalaren version.
    """
    s = 1./(1. + np.exp(-x))
    return s

#Um die Wahrscheinlichkeit für alle Zuordnungen zu berechnen, unter Berücksichtigung von 2 Merkmalen
def p(X, w1, w2, b):
    """Wahrscheinlichkeit dass ein Hunde krank ist fuer alle Hunde gleichzeitig.
    Args:
        X (numpy.ndarray): Datenmatrix:
            - pro Hund eine Zeile (erster Index)
            - pro Eigenschaft eine Spalte (zweiter Index)
        w1 (float): Gewichtet die Herzfrequenz.
        w2 (float): Gewichtet die ST-Depression
        b (float): Verschiebung zur Unterscheidung des Gesundheitszustands.
        Returns:
            numpy.ndarray: Vektor:
                - pro Hund eine Wahrscheinlichkeit, dass der Hund krank ist
                    (Werte < 0.5: Hund ist wahrscheinlich gesund,
                     Werte > 0.5 Hund ist wahrcheinlich krank.)
    """
    l = X[:, 0]*w1 + X[:, 1]*w2 + b
    v = sigmoid(l)
    return v

#Um das Minimum zu berechnen
def derivative(D, w1, w2, b):
    """Ableitung der Gewichte und des Bias.
    Args:
        D (numpy.ndarray): Datenmatrix:
            - pro Hund eine Zeile (erster Index)
            - pro Eigenschaft eine Spalte (zweiter Index)
        w1 (float): Gewichtet die Herzfrequenz.
        w2 (float): Gewichtet die ST-Depression
        b (float): Verschiebung zur Unterscheidung des Gesundheitszustands.
        Returns:
            (float, float, float): Die neuen Gweichte (w1, w2) und den neuen Bias (b).
    """

    labels = D[:, -1]
    X = D[:, [7, 9]]

    d_w1 = np.mean((p(X, w1, w2, b) - labels) * X[:, 0])
    d_w2 = np.mean((p(X, w1, w2, b) - labels) * X[:, 1])
    d_b = np.mean((p(X, w1, w2, b) - labels))
    return d_w1, d_w2, d_b

#Um einen Vergleich zw. Vorhersage und den vorliegenden Datenpunkten anzustellen
def accuracy(Y, Y_hat):
    """Accuracy der Predictions.
    Args:
        Y (numpy.ndarray): Vektor mit den Lables:
            - pro Hund ein Wert mit dem tatsaechlichem
                gesundheitszustand des Hundes.
        Y_hat (numpy.ndarray): Vektor mit Predictions (0 oder 1) für jeden Hund.
        Returns:
            (float): Accuracy.
    """
    return np.mean(Y == Y_hat)

---

### Tipps & Tricks (nicht für Lösung im KI Kurs notwendig)

Erneut können wir die Funktionen nicht "einfach so" testen, aber wir können uns für den Test wieder temporär eigene Daten ausdenken und diese in die Funktionen einsetzen. Somit erhalten wir einen ersten Eindruck über den Output unserer Funktionen. Das geht z.B. so:

In [8]:
#Sigmoid
print(sigmoid(-10), sigmoid(0.05), sigmoid(10))

4.5397868702434395e-05 0.5124973964842103 0.9999546021312976


In [9]:
#p
X = D[:, [7, 9]]
p(X,0.01,0.01,0.01)[:10] #zeige nur die ersten 10 Einträge

array([0.82244483, 0.75119578, 0.79017822, 0.87157983, 0.85119443,
       0.8579053 , 0.83834971, 0.83834971, 0.81667789, 0.83075675])

In [10]:
#derivative
derivative(D,0.01,0.01,0.01)

(36.99401820361357, 0.052416868246057534, 0.20793499726578424)

In [11]:
#accuracy

#1. imaginäres Beispiel (identisch zu oben)
preds_bad_temporary = np.array([0.7,
                               0.4,
                               0.0,
                               0.9])

preds_good_temporary = np.array([0.9,
                                0.1,
                                0.9,
                                0.0])

# wichtig: aufrunden!
preds_bad_temporary = np.round(preds_bad_temporary)
preds_good_temporary = np.round(preds_good_temporary)

labels_temporary = np.array([1.0,
                            0.0,
                            1.0,
                            0.0])

acc_bad = accuracy(preds_bad_temporary.round(),labels_temporary)
acc_good = accuracy(preds_good_temporary,labels_temporary)

#Zur verbesserten Übersicht lassen wir uns zu unserer Ausgabe eine Zuordnung (Bsp.1) mit ausgeben
print("Bsp. 1:")
#Zum besseren und direkten Verständnis
print("Genauigkeit der schlechten Vorhersagen: {}\n Genauigkeit der guten Vorhersagen: {}\n".format(acc_bad,acc_good))
#Zur besseren Strukturierung unserer Ausgabe lassen wir uns eine gestrichelte Linie ausgeben
print("-"*20)

#2. Beispiel mit obiger Funktion p

preds = np.round(p(X,0.01,0.01,0.01)) #Runden nicht vergessen!
labels = D[:, -1]
acc = accuracy(preds,labels)

print("Bsp.2:")
print("Genauigkeit der Vorhersagen: {}".format(acc))
print("-"*20)

Bsp. 1:
Genauigkeit der schlechten Vorhersagen: 0.5
 Genauigkeit der guten Vorhersagen: 1.0

--------------------
Bsp.2:
Genauigkeit der Vorhersagen: 0.596244131455399
--------------------


Super jetzt hast du getestet, dass die von dir gecodeten Funktionen klappen!

---

Deine Aufgabe ist es mittels des Gradientenverfahrens die Gewichte und den Bias zu finden, welche uns die besten Diagnosen liefern. Damit wir kein globales Minimum übersehen (zu große Lernrate) oder gar nicht erst zu einem globalen Minimum kommen (zu kleine Lernrate), müssen wir die Änderungen unserer Parameter schrittweise, mittels der **Step-Funktion** vornehmen. Implementiere die Step-Funktion (13/15). Dies ist die MUSTERLÖSUNG! Die Coding-Konsole im KI-Kurs enthält weitere Informationen zu Funktionen, Argumenten oder Erklärungen zum Ergebnis.

In [12]:
#import numpy as np

#Sigmoid Funktion - wurde berits oben definiert
#def sigmoid(x):
    #s = 1./(1. + np.exp(-x))
    #return s

#Wahrscheinlichkeit - wurde berits oben definiert
#def p(X, w1, w2, b):
    #l = X[:, 0]*w1 + X[:, 1]*w2 + b
    #v = sigmoid(l)
    #return v

#Fkt. derivative - wurde bereits oben implementiert, wird jedoch um labels erweitert    
def derivative(X, labels, w1, w2, b):
    d_w1 = np.mean((p(X, w1, w2, b) - labels) * X[:, 0])
    d_w2 = np.mean((p(X, w1, w2, b) - labels) * X[:, 1])
    d_b = np.mean((p(X, w1, w2, b) - labels))
    return d_w1, d_w2, d_b
    
    labels = D[:, -1]
    X = D[:, [7, 9]]


#Genauigkeit - wurde beireits im letzten Schritt implementiert
#def accuracy(Y, Y_hat):
    #return np.mean(Y == Y_hat)

def step(D, w1, w2, b, alpha):
    """Durchfuehren eines Optimierungsschritts.
    Args:
        D (numpy.ndarray): Datenmatrix:
            - pro Hund eine Zeile (erster Index)
            - pro Eigenschaft eine Spalte (zweiter Index)
        w1 (float): Gewichtet die Herzfrequenz.
        w2 (float): Gewichtet die ST-Depression
        b (float): Verschiebung zur Unterscheidung des Gesundheitszustands.
        alpha (float): Learning rate.
        Returns:
            (float, float, float): Die neuen Gewichte (w1, w2) und den neuen Bias (b).
    """
    labels = D[:, -1]
    X = D[:, [7, 9]]

    d_w1, d_w2, d_b = derivative(D, labels, w1, w2, b)
                                 
    w1_neu = w1 - alpha * d_w1
    w2_neu = w2 - alpha * d_w2
    b_neu = b - alpha * d_b
    return w1_neu, w2_neu, b_neu

---

### Tipps & Tricks (nicht für Lösung im KI Kurs notwendig)

Schön und gut, die Step-Funktion ist nun implementiert. Aber wie können wir uns anschauen, was sie bewirkt und ob wir alles richtig gemacht haben?

In [13]:
#Teste deine Implementierung
#Setze im ersten Schritt alle Gewichte und den Bias auf 0.1 sowie die Lernrate auf alpha=0.0001
print(step(D,0.1, 0.1,0.1,0.0001))

(0.09983921595658289, 0.09999402189422861, 0.09998322876865283)


Der Optimierungsschritt wurde durchgeführt! Die Ausgabe zeigt dir die neu geschätzten Gewichte.

---

Die nächste Aufgabe des 4. Kapitels (Abschnitt 14/15), werden wir schrittweise bearbeiten. Wir starten mit Schritt 1, bei dem wir die bestehende Modellfunktion p() an ein multivariates-Modell anpassen müssen. Das bedeutet, dass jetzt alle 13 Merkmale verwendet werden, um den Gesunheitszustand eines Hundes (14. Merkmal) vorherzusagen. Es entstehen Gewichts- und Merkmalsvektoren, für deren Verrechnung wir wieder die numpy Funktion np.dot verwenden (Reminder - Skalarprodukt):

In [14]:
#import numpy as np

def sigmoid(x):
    s = 1./(1. + np.exp(-x))
    return s

def p(X, w):
    l = np.dot(X, w)
    v = sigmoid(l)
    return v

Schritt 2. (Abschnitt 14/15) nun passen wir auch das Gradientenabstiegsverfahren für alle Merkmale an:

In [15]:
def derivative(X, labels, w):
    d = np.mean((p(X, w) - labels)[:, None] * X, axis=0)
    return d

Im Schritt 3(a) & 3(b) des (Abschnitt 14/15), erweitern wir nun auch die Step Funktion auf alle Merkmale:

In [16]:
#3a erweitern der Step Funktion
def step(D, w, alpha):
    labels = D[:, -1]
    X = np.ones((D.shape[0], D.shape[1]))
    
    X[:, :-1] = D[:, :-1] #Erklärung
    
    d_w = derivative(X, labels, w) #Erklärung
    
    w_neu = w - alpha * d_w #3b Gewichtsvektor aktualisieren
    return w_new

---


In der letzten Aufgabe des vierten Kapitels geht es um die Optimierung, also die Steigerung der Genauigkeit unseres Multivariaten Modells (Abschnitt 15/15). Wir werden die Funktion optimierung() implementieren, die unsere Parameter (Gewichte, Bias) so lange aktualisiert, bis sich unsere Vorhersage nicht mehr verbessert. Dies können wir mit der Implementierung einer for-Schleife bewirken, die abbricht (break) sofern der Unterschied sehr gering ausfällt z.B. < 1eˆ-6 ist. Dies ist die MUSTERLÖSUNG! Die Coding-Konsole im KI-Kurs enthält weitere Informationen zu Funktionen, Argumenten oder Erklärungen zum Ergebnis.

In [17]:
#import numpy as np


def sigmoid(x):
    s = 1./(1. + np.exp(-x))
    return s

def p(X, w):
    l = np.dot(X, w)
    v = sigmoid(l)
    return v



def derivative(X, Y, w):
    d = np.mean((p(X, w) - Y)[:, None] * X, axis=0)
    return d


def accuracy(Y, Y_hat):
    return np.mean(Y == Y_hat)

def cost(predictions, labels):
    
    eps = np.finfo(float).eps # avoid dividing by zero in log
    
    predictions[predictions==0] = 1e-16
    predictions[predictions==1] = 1.-1e-16   
    c = np.mean(-(labels*np.log(predictions)
          + (1-labels)*np.log(1-predictions + eps)))
    return c

def step(D, w, alpha):
    X = np.ones((D.shape[0], D.shape[1]))
    X[:, 1:] = D[:, :-1]
    Y = D[:, -1]
    
    labels = D[:, -1]
    d_w = derivative(X, Y, w)
    w_new = w - alpha * d_w
    return w_new

def vorhersage(X, w):
    Y_hat = np.round((p(X, w) > 0.5))
    return Y_hat

def optimierung(D):
    w = np.random.normal(size=14)*0.01
    alpha = 0.0001

    X = np.ones((D.shape[0], D.shape[1]))
    X[:, 1:] = D[:, :-1]

    last_cost = np.inf
    for i in range(50000):
        w = step(D, w=w, alpha=alpha)
        current_cost = cost(p(X, w), D[:, -1])
        if (np.abs(last_cost - current_cost) < 1e-6):
            break
        last_cost = current_cost
    Y_hat = vorhersage(X, w)
    print("DONE", i, accuracy(D[:, -1], Y_hat))
    return Y_hat

Puuuuh - Im Fenster des KI Kurses wird keine Fehlermeldung angezeigt. Yeei! ... Aber was genau bringt uns dieser lange Code-Block? Im nächsten Abschnitt testen wir unser Multivariates Modell und schauen uns an, wie genau die Vorhersage unseres Modells mittels der Optimierungsfunktion werden kann. Bereit ?!

---

## Tipps & Tricks (nicht für Lösung im KI Kurs notwendig)

### Ein Multivariates Modell

Nun haben wir unser Modell Schritt für Schritt erweitert und sind bei einem multivariaten Modell angelangt, welches alle 13 Variablen in die Vorhersage von gesund oder krank mit einbezieht. Zusätzlich haben wir im letzten Schritt eine Optimierungsfunktion eingebaut, damit die Vorhersagen unseres Modells so genau wie möglich sind. 

<br> Jetzt wollen wir das Ganze natürlich auch mal testen und schauen, welche Genauigkeit unser Modell erzielen kann. Dieser Teil ist **nicht für die Lösung des KI Kurses nötig**, gibt dir allerdings ein besseres Verständnis für die Funktionen und dein multivariates Modell. Wenn du noch nicht alle Schritte im Code verstehst, macht das überhaupt nichts! In Kapitel 5 gehen wir fundierter auf das Trainieren von komplexen Modellen ein.

Die Funktion optimierung() erledigt den Großteil der Arbeit für uns. Schaut euch die #Kommentare an, damit ihr jeden einzelnen Schritt nachverfolgen könnt:


In [18]:
#Wir werfen erst mal einen Blick in unseren Datensatz
D[:, :-1] 

array([[13.        ,  1.        ,  1.        , ...,  3.        ,
         0.        ,  6.        ],
       [14.33333333,  1.        ,  4.        , ...,  2.        ,
         3.        ,  3.        ],
       [14.33333333,  1.        ,  4.        , ...,  2.        ,
         2.        ,  7.        ],
       ...,
       [16.        ,  1.        ,  3.        , ...,  2.        ,
         2.        ,  5.78873239],
       [16.33333333,  0.        ,  3.        , ...,  1.        ,
         1.6       ,  3.        ],
       [16.66666667,  1.        ,  2.        , ...,  1.        ,
         1.6       ,  5.78873239]])

In [19]:
np.ones((D.shape[0], D.shape[1])).shape

(426, 14)

In [20]:
def optimierung(D):
    w = np.random.normal(size=14)*0.01 # Initialisierung der Gewichte
    alpha = 0.0001 # Festlegen der Lernrate

    X = np.ones((D.shape[0], D.shape[1])) # wir generieren ein Array mit der gleichen Größe wie D und füllen es mit 1en auf
    X[:, 1:] = D[:, :-1] # wir übernehmen alle Werte bis auf die Labels (die soll unser Klassifikator ja nicht sehen) in unser neues Array
    Y = D[:, -1]
    
    last_cost = np.inf # optimale Kosten tracken
    for i in range(50000): #50000 Epochen iterieren lassen
        w_neu = step(D, w=w, alpha=alpha) # einen Trainingsschritt ausführen
        w = w_neu
        
        current_cost = cost(p(X, w), D[:, -1])  # aktuelle Kosten berechnen
        if (np.abs(last_cost - current_cost) < 1e-6): # wenn Kosten nicht mehr sinken -> abbrechen (early stopping)
            break
        last_cost = current_cost # Kosten aktualisieren (nur zum Tracken)
    Y_hat = vorhersage(X, w)  # ganz am Ende Vorhersage machen
    print("DONE", i, accuracy(Y, Y_hat)) #Ausgabe
    return Y_hat # Vorhersage zurückgeben

Unsere Optimierungs Funktion trainiert unser Modell bis sich unsere Vorhersage nicht mehr verbessert. Am Ende speichern wir die Vorhersagen in der Variable preds ab. Alles was wir dann noch tun müssen ist, unsere Kosten sowie die Genauigkeit zu berechnen und hierfür müssen wir preds mit den labels (Originaldaten) vergleichen:

In [21]:
#Abspeichern der Variablen, um diese der Kostenfunktion zu übergeben
preds = optimierung(D)
labels =  D[:, -1]

DONE 31153 0.8403755868544601


In [22]:
Y = D[:, -1]
accuracy(preds,Y)

0.8403755868544601

Unser Modell erzielt eine Genauigkeit von 84%? Na wer sagts denn? :D

**Hinweis**: Du erhälst eine Fehlermeldung?
    Aktuell liefert uns unsere vorhersage-Funktion booleans, also Wahrheitswerte zurück.
    Diese werden von unserer Kostenfunktion nicht richtig interpretiert und deshalb tritt hier eine Fehlermeldung auf.
    Wir können diesen Fehler beheben, indem wir unsere vorhersage-Funktion anpassen und Gleitkommazahlen statt Booleans
    zurückgeben:

In [23]:
def vorhersage(X, w):
    Y_hat = (p(X, w) > 0.5)
    return Y_hat.astype("float64")

Nun tritt auch keine Fehlermeldung auf:

In [24]:
cost(preds, labels)

/tmp/ipykernel_100/1499381281.py:29: RuntimeWarning: divide by zero encountered in log
  c = np.mean(-(labels*np.log(predictions)
/tmp/ipykernel_100/1499381281.py:29: RuntimeWarning: invalid value encountered in multiply
  c = np.mean(-(labels*np.log(predictions)
/tmp/ipykernel_100/1499381281.py:30: RuntimeWarning: divide by zero encountered in log
  + (1-labels)*np.log(1-predictions + eps)))
/tmp/ipykernel_100/1499381281.py:30: RuntimeWarning: invalid value encountered in multiply
  + (1-labels)*np.log(1-predictions + eps)))


nan

In [25]:
accuracy(preds,labels)

0.8403755868544601

---

### Du hast es geschafft! 🥳 

Auch das vierte Kapitel haben wir erfolgreich Überstanden! 💪🏼 Das war mit Abstand das mathematischste Kapitel dieses Tutorials - Super gemacht! <br> Mach eine kurze oder längere Pause! <br> Denn ... 
jetzt geht es daran ein neuronales Netz zu programmieren ... 😎